In [95]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

# Reading CSV file and creating DataFrame
df = pd.read_csv("training_data.csv")
df.head()

,Unnamed: 0,Rank,Date,Title,Danceability,Energy,Loudness,Speechiness,Acousticness,Instrumentalness,Valence,id
0,0,1,29/05/2023,Ella Baila Sola,0.668,0.758,-5176.0,0.033,0.483,0.000,0.834,3qQbCzHBycnDpGskqOWY0E
1,2,2,29/05/2023,WHERE SHE GOES,0.652,0.800,-4019.0,0.061,0.143,0.629,0.234,7ro0hRteUMfnOioTFI5TG1
2,3,3,29/05/2023,La Bebe - Remix,0.812,0.479,-5678.0,0.333,0.213,0.000,0.559,2UW7JaomAMuX9pZrjVpHAU
3,5,4,29/05/2023,Cupid - Twin Ver.,0.783,0.592,-8332.0,0.033,0.435,0.000,0.726,7FbrGaHYVDmfr7KoLIZnQ7
4,6,5,29/05/2023,un x100to,0.569,0.724,-4076.0,0.047,0.228,0.000,0.562,6pD0ufEQq0xdHSsRbg9LBK


In [96]:
# Dropping collumns not needed to train our model
df = df.drop(columns=["Date","Title","id"])
df.head()


,Unnamed: 0,Rank,Danceability,Energy,Loudness,Speechiness,Acousticness,Instrumentalness,Valence
0,0,1,0.668,0.758,-5176.0,0.033,0.483,0.000,0.834
1,2,2,0.652,0.800,-4019.0,0.061,0.143,0.629,0.234
2,3,3,0.812,0.479,-5678.0,0.333,0.213,0.000,0.559
3,5,4,0.783,0.592,-8332.0,0.033,0.435,0.000,0.726
4,6,5,0.569,0.724,-4076.0,0.047,0.228,0.000,0.562


In [97]:
#Binning our data

# Defining bin edges and labels
bins = [0, 50, 100, 150, 200]
labels = [1, 2, 3, 4]

#Creating column with binned data
df['Popularity'] = pd.cut(df['Rank'], bins=bins, labels=labels, right=True)



In [98]:
#Dropping 'Rank' and unnamed column
df = df.drop(columns="Rank")
df = df.drop(columns="Unnamed: 0")

df.head()

,Danceability,Energy,Loudness,Speechiness,Acousticness,Instrumentalness,Valence,Popularity
0,0.668,0.758,-5176.0,0.033,0.483,0.000,0.834,1
1,0.652,0.800,-4019.0,0.061,0.143,0.629,0.234,1
2,0.812,0.479,-5678.0,0.333,0.213,0.000,0.559,1
3,0.783,0.592,-8332.0,0.033,0.435,0.000,0.726,1
4,0.569,0.724,-4076.0,0.047,0.228,0.000,0.562,1


In [99]:
df.dtypes

Danceability         float64
Energy               float64
Loudness             float64
Speechiness          float64
Acousticness         float64
Instrumentalness     float64
Valence              float64
Popularity          category
dtype: object

In [100]:
#Removing 'Rank' from features data
y = df['Popularity']
X = df.drop(columns="Popularity")

In [101]:
# get_dummies
y = pd.get_dummies(y)

y.head()

,1,2,3,4
0,True,False,False,False
1,True,False,False,False
2,True,False,False,False
3,True,False,False,False
4,True,False,False,False


In [102]:
#Splitting into training/testing datasets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)

In [103]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [107]:
# Defining our model
num_input_features = len(X_train_scaled[0])
hidden_layer_nodes_1 = 5
hidden_layer_nodes_2 = 2

nn = tf.keras.models.Sequential()

# Creating first hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_layer_nodes_1, input_dim=num_input_features, activation="relu"))

# Creating second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_layer_nodes_2, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=4, activation="sigmoid"))

# Check the structure of the model
nn.summary()

/Users/CaelWillis/anaconda3/envs/dev/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_21 (Dense)                │ (None, 5)              │            40 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_22 (Dense)                │ (None, 2)              │            12 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_23 (Dense)                │ (None, 4)              │            12 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 64 (256.00 B)

 Trainable params: 64 (256.00 B)

 Non-trainable params: 0 (0.00 B)

In [108]:
# Compiling and training model
nn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
fit_model = nn.fit(X_train_scaled,y_train,epochs=100)

Epoch 1/100
10987/10987 ━━━━━━━━━━━━━━━━━━━━ 5s 411us/step - accuracy: 0.2606 - loss: 0.5734
Epoch 2/100
10987/10987 ━━━━━━━━━━━━━━━━━━━━ 4s 402us/step - accuracy: 0.2763 - loss: 0.5607
Epoch 3/100
10987/10987 ━━━━━━━━━━━━━━━━━━━━ 4s 387us/step - accuracy: 0.2771 - loss: 0.5604
Epoch 4/100
10987/10987 ━━━━━━━━━━━━━━━━━━━━ 4s 406us/step - accuracy: 0.2776 - loss: 0.5603
Epoch 5/100
10987/10987 ━━━━━━━━━━━━━━━━━━━━ 4s 382us/step - accuracy: 0.2777 - loss: 0.5603
Epoch 6/100
10987/10987 ━━━━━━━━━━━━━━━━━━━━ 4s 391us/step - accuracy: 0.2767 - loss: 0.5601
Epoch 7/100
10987/10987 ━━━━━━━━━━━━━━━━━━━━ 5s 446us/step - accuracy: 0.2787 - loss: 0.5597
Epoch 8/100
10987/10987 ━━━━━━━━━━━━━━━━━━━━ 5s 464us/step - accuracy: 0.2800 - loss: 0.5594
Epoch 9/100
10987/10987 ━━━━━━━━━━━━━━━━━━━━ 5s 466us/step - accuracy: 0.2792 - loss: 0.5595
Epoch 10/100
10987/10987 ━━━━━━━━━━━━━━━━━━━━ 5s 472us/step - accuracy: 0.2793 - loss: 0.5594
Epoch 11/100
10987/10987 ━━━━━━━━━━━━━━━━━━━━ 5s 483us/step - accurac